In [1]:
%matplotlib inline

使用torch.nn包来构建神经网络。

上一讲已经讲过了autograd，nn包依赖autograd包来定义模型并求导。 一个nn.Module包含各个层和一个forward(input)方法，该方法返回output。

例如：
它是一个简单的前馈神经网络，它接受一个输入，然后一层接着一层地传递，最后输出计算的结果。

神经网络的典型训练过程如下：

定义包含一些可学习的参数(或者叫权重)神经网络模型；
在数据集上迭代；
通过神经网络处理输入；
计算损失(输出结果和正确值的差值大小)；
将梯度反向传播回网络的参数；
更新网络的参数，主要使用如下简单的更新原则：
weight = weight - learning_rate * gradient

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [39]:
class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()
        #1 1 input image channel,6 output channels,5X5 square convolution
        #kernel
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        #an affine operation:y = wx+b
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        #MAX pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        # If the size is a square you can only specify a single number
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self,x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [40]:
params = list(net.parameters())
print(len(params))
print(params[0].size())# conv1 weight
print(params[1].size())
print(params)

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
[Parameter containing:
tensor([[[[-0.1777,  0.1334,  0.1928,  0.1886, -0.0259],
          [-0.0640,  0.1965, -0.0622,  0.0577,  0.0441],
          [-0.1418, -0.1732, -0.0646, -0.0613,  0.0869],
          [-0.0068,  0.1365,  0.1388,  0.0300,  0.0525],
          [ 0.0023,  0.0736, -0.1550, -0.1168,  0.0338]]],


        [[[-0.0340,  0.0732,  0.0102, -0.1594, -0.0181],
          [-0.1757, -0.0435,  0.1386,  0.0353,  0.0996],
          [ 0.0406, -0.1572, -0.0910, -0.0728,  0.1855],
          [-0.1952,  0.0627, -0.1137, -0.0414, -0.0563],
          [-0.0475, -0.1039,  0.1167,  0.1392,  0.1661]]],


        [[[-0.0891,  0.0264,  0.0981,  0.1642,  0.1386],
          [ 0.1290,  0.1251, -0.1723,  0.0739, -0.1682],
          [-0.1590,  0.0480,  0.1336,  0.0448,  0.1866],
          [-0.1877,  0.0865,  0.1541,  0.0366, -0.0533],
          [-0.1393, -0.0767, -0.0869,  0.1764,  0.1041]]],


        [[[-0.1041, -0.1756, -0.0099,  0.1324, -0.0711],
         

测试随机输入32×32。 注：这个网络（LeNet）期望的输入大小是32×32，如果使用MNIST数据集来训练这个网络，请把图片大小重新调整到32×32。

In [41]:
input = torch.randn(1,1,32,32)
input

tensor([[[[ 1.3786, -0.1685, -1.5023,  ...,  0.1058, -1.8240, -0.1033],
          [ 0.8130, -1.4512,  0.5194,  ..., -0.4201,  0.1843, -0.4848],
          [ 0.4310,  1.5333,  1.6982,  ..., -0.4078, -0.4040,  1.1970],
          ...,
          [ 1.0566,  0.6966,  1.5472,  ...,  1.0282,  0.0402, -2.0220],
          [-1.2033, -1.2452, -0.2053,  ...,  0.6585, -0.5915,  0.7812],
          [ 1.6567, -1.8875,  0.2209,  ...,  1.1646,  1.6353,  1.2260]]]])

In [42]:
input.size()

torch.Size([1, 1, 32, 32])

In [43]:
out = net(input)
print(out)

tensor([[-0.0851,  0.0041, -0.1146, -0.0295, -0.1480,  0.0410,  0.0484,  0.2101,
         -0.0708, -0.1421]], grad_fn=<AddmmBackward0>)


将所有参数的梯度缓存清零，然后进行随机梯度的反向传播

In [44]:
net.zero_grad()
out.backward(torch.randn(1,10))

``torch.nn`` 只支持小批量输入。整个 ``torch.nn`` 包都只支持小批量样本，而不支持单个样本。
例如，``nn.Conv2d`` 接受一个4维的张量，
``每一维分别是sSamples * nChannels * Height * Width（样本数*通道数*高*宽）``。

如果你有单个样本，只需使用 ``input.unsqueeze(0)`` 来添加其它的维数

在继续之前，我们回顾一下到目前为止用到的类。

回顾:

torch.Tensor：一个用过自动调用 backward()实现支持自动梯度计算的 多维数组 ， 并且保存关于这个向量的梯度 w.r.t.
nn.Module：神经网络模块。封装参数、移动到GPU上运行、导出、加载等。
nn.Parameter：一种变量，当把它赋值给一个Module时，被 自动 地注册为一个参数。
autograd.Function：实现一个自动求导操作的前向和反向定义，每个变量操作至少创建一个函数节点，每一个Tensor的操作都回创建一个接到创建Tensor和 编码其历史 的函数的Function节点。
重点如下：

定义一个网络
处理输入，调用backword
还剩：

计算损失
更新网络权重
损失函数
一个损失函数接受一对 (output, target) 作为输入，计算一个值来估计网络的输出和目标值相差多少。

*译者注：output为网络的输出，target为实际值*

nn包中有很多不同的损失函数。 nn.MSELoss是一个比较简单的损失函数，它计算输出和目标间的均方误差， 例如：

In [45]:
output = net(input)
target = torch.randn(10) #随机值作为样例
target = target.view(1,-1)#使target和output的shape相同
criterion = nn.MSELoss()
loss = criterion(output,target)
loss

tensor(0.6502, grad_fn=<MseLossBackward0>)

现在，如果在反向过程中跟随loss ， 使用它的 .grad_fn 属性，将看到如下所示的计算图。
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
      所以，当我们调用 loss.backward()时,整张计算图都会 根据loss进行微分，而且图中所有设置为requires_grad=True的张量 将会拥有一个随着梯度累积的.grad 张量。

为了说明，让我们向后退几步:

In [46]:
print(loss.grad_fn)#MSEloss

In [47]:
print(loss.grad_fn.next_functions[0][0])#Linear

In [49]:
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])#Relu

反向传播
调用loss.backward()获得反向传播的误差。

但是在调用前需要清除已存在的梯度，否则梯度将被累加到已存在的梯度。

现在，我们将调用loss.backward()，并查看conv1层的偏差（bias）项在反向传播前后的梯度。

In [50]:
net.zero_grad()

print('conv1.bias.grad berore backward')
print(net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad berore backward')
print(net.conv1.bias.grad)

conv1.bias.grad berore backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad berore backward
tensor([-0.0131, -0.0065, -0.0041,  0.0129,  0.0012,  0.0019])


更新权重
在实践中最简单的权重更新规则是随机梯度下降（SGD）：

 ``weight = weight - learning_rate * gradient``

我们可以使用简单的Python代码实现这个规则：

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
但是当使用神经网络是想要使用各种不同的更新规则时，比如SGD、Nesterov-SGD、Adam、RMSPROP等，PyTorch中构建了一个包torch.optim实现了所有的这些规则。 使用它们非常简单：

In [51]:
import torch.optim as optim

In [52]:
#create your optimizer
optimizer = optim.SGD(net.parameters(),lr=0.01)

#in your training loop
optimizer.zero_grad() #zero the gradient buffers
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step()# Does the update


.. 注意::

  观察如何使用``optimizer.zero_grad()``手动将梯度缓冲区设置为零。
  这是因为梯度是按Backprop部分中的说明累积的。